<a href="https://colab.research.google.com/github/derrodo61/notebooks/blob/main/diffusers_effective.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**More efficiency by:**

- using float16
- pipeline.to("cuda")
- use a more effective scheduler
- reduce number of inference steps


In [ ]:
!pip install diffusers torch

In [ ]:
from diffusers import DiffusionPipeline
import torch

#model_id = "runwayml/stable-diffusion-v1-5"
model_id = "SG161222/Realistic_Vision_V6.0_B1_noVAE"

pipeline = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=False,
    safety_checker = None,
    requires_safety_checker = False
    )
pipeline = pipeline.to("cuda")

In [3]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
prompt = "esthetic portrait of an beautiful woman"
generator = torch.Generator("cuda").manual_seed(0)

# generate 1 image

image = pipeline(prompt, generator=generator, num_inference_steps=20).images[0]

#show it

image

In [5]:
# show valid schedulers for this pipeline
pipeline.scheduler.compatibles

[diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
 diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
 diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
 diffusers.schedulers.scheduling_pndm.PNDMScheduler,
 diffusers.schedulers.scheduling_deis_multistep.DEISMultistepScheduler,
 diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_heun_discrete.HeunDiscreteScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2DiscreteScheduler,
 diffusers.schedulers.scheduling_unipc_multistep.UniPCMultistepScheduler,
 diffusers.schedulers.scheduling_edm_euler.EDMEulerScheduler,
 diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
 diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2Ancestr

In [6]:
def get_inputs(batch_size=1):
    generator = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]
    prompts = batch_size * [prompt]
    num_inference_steps = 25

    return {"prompt": prompts, "generator": generator, "num_inference_steps": num_inference_steps}

In [53]:
from diffusers.utils import make_image_grid

prompt = "esthetic portrait of an beautiful woman"
images = pipeline(**get_inputs(batch_size=4)).images

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
make_image_grid(images, 1, 4)